Dependencies

In [ ]:
from __future__ import print_function

#!pip install numpy==1.19.5
#!pip install tensorflow==2.2.0
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

#from tensorflow import keras
### hack tf-keras to appear as top level keras
#import sys
#sys.modules['keras'] = keras
### end of hack

from keras.layers import Input, Dense, Lambda, Reshape
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()



print(np.__version__)

Global Variables

In [ ]:
# defining the key parameters
batch_size = 20

IMG_SIZE = 80 # default 236
# Parameters of the input images (handwritten digits)
# 236 x 236
original_dim = IMG_SIZE*IMG_SIZE

# Latent space is of dimension 2.  This means that we are reducing the dimension from 784 to 2
latent_dim = 2
second_dim = 256
intermediate_dim = 640
epochs = 200
epsilon_std = 1.0

Data Loading

In [ ]:
import os
import cv2  # pip3 install opencv-python


i = 0
j = 0
path = "images"
images = []
y =[]
for img in os.listdir(path):
    try:
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        if j == 199:
            plt.imshow(img_array, cmap="gray")
            plt.show()
        new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
        if j != 134 and j!=155 and j!=363 and j!=366:
            images.append(new_array)
            y.append(j)
        j+=1
    except Exception as e:
        pass
    # plt.imshow(img_array, cmap="gray")
    # plt.show()
    # i +=1
    # if i == 0:
    #     break
print(len(images))    

# new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
# plt.imshow(new_array, cmap="gray")
# plt.show()


# j = 0
# for image in new_array:
#     images.append(image)
#     y.append(j)
#     j+=1




In [ ]:
X = np.array(images).reshape(-1,IMG_SIZE,IMG_SIZE,1)

x_train = X 
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
y_train = y


# x_test = np.array(images[301:]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
# x_test = x_test.astype('float32') / 255.
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
# print(len(x_test))
# y_test = list(range(128))

x_test = x_train
y_test = y

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(y_train)

#x_train = x_train.astype('float32') / 255.
#x_test = x_test.astype('float32') / 255.
#x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
#x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

Sampling Definition

In [ ]:
def sampling(args: tuple):
    # we grab the variables from the tuple
    z_mean, z_log_var = args
    print(z_mean)
    print(z_log_var)
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon  # h(z)

Encoder

In [ ]:
# input to our encoder
x = Input(shape=(original_dim,), name="input")
# intermediate layer
h = Dense(intermediate_dim, activation='relu', name="encoding")(x)

h = Dense(second_dim, activation='relu', name="encoding2")(h)

# defining the mean of the latent space
z_mean = Dense(latent_dim, name="mean")(h)
# defining the log variance of the latent space
z_log_var = Dense(latent_dim, name="log-variance")(h)
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
# defining the encoder as a keras model
encoder = Model(x, [z_mean, z_log_var, z], name="encoder")
# print out summary of what we just did
encoder.summary()

Decoder

In [ ]:
# Input to the decoder
input_decoder = Input(shape=(latent_dim,), name="decoder_input")
# taking the latent space to intermediate dimension
decoder_h = Dense(second_dim, activation='relu', name="decoder_h2")(input_decoder)
decoder_h = Dense(intermediate_dim, activation='relu', name="decoder_h")(decoder_h)
# decoder_h = Dense(intermediate_dim, activation='relu', name="decoder_h")(input_decoder)

# getting the mean from the original dimension
x_decoded = Dense(original_dim, activation='sigmoid', name="flat_decoded")(decoder_h)
# defining the decoder as a keras model
decoder = Model(input_decoder, x_decoded, name="decoder")
decoder.summary()

VAE Initialization

In [ ]:
# grab the output. Recall, that we need to grab the 3rd element our sampling z
output_combined = decoder(encoder(x)[2])
# link the input and the overall output
vae = Model(x, output_combined)
# print out what the overall model looks like
vae.summary()

VAE Loss Definition

In [ ]:
import tensorflow as tf
from keras.datasets import mnist
from keras import metrics

def vae_loss(x: tf.Tensor, x_decoded_mean: tf.Tensor):
  # Aca se computa la cross entropy entre los "labels" x que son los valores 0/1 de los pixeles, y lo que salió al final del Decoder.
  xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean) # x-^X
  kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
  vae_loss = K.mean(xent_loss + kl_loss)
  return vae_loss

vae.compile( loss=vae_loss)
vae.summary()

VAE Training

In [ ]:
vae.fit(x_train, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)


2D Map Visualization

In [ ]:


x_test_encoded = encoder.predict(x_test, batch_size=batch_size)[0]
i = 0


plt.figure(figsize=(6, 6))

plt.scatter(x_test_encoded[:,0], x_test_encoded[:,1], c=y_test, cmap='viridis')
# for xi in x_test_encoded:
#     plt.text(xi[0], xi[1], i)
#     if xi[1]> -12:
#         print(xi)
#     i += 1
# plt.text(x_test_encoded[:,0], x_test_encoded[:,1], y_test)
plt.colorbar()
plt.show()

In [ ]:
sample_vector = np.array([[-4,-2.5]])
decoded_example = decoder.predict(sample_vector)
decoded_example_reshaped = decoded_example.reshape(80, 80)
plt.imshow(decoded_example_reshaped, cmap='Greys_r')

VAE Generation

In [ ]:
n = 5  # figure with 15x15 digits
digit_size = 80
figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))
print(grid_y)
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()

In [ ]:
print(grid_x)

In [ ]:
print(z_sample)